## NORC County Analysis and Pattern-Based Clustering

This notebook implements a comprehensive data processing pipeline for analyzing county-level socioeconomic and health indicators from the NORC (National Opinion Research Center) dataset, combined with patterns generated from the AK Analyst Platform.

## Project Overview

The pipeline processes county-level data through several stages:
1. Feature Dictionary Creation
2. Pattern Constraint Processing
3. County-Pattern Matching
4. Data Normalization
5. Final Dataset Generation

## Data Sources

- **NORC Dataset (2017-2020)**: Contains county-level socioeconomic and health indicators including:
  - Population demographics
  - Education levels
  - Economic indicators
  - Health facility access
  - Social resilience scores
  - Death rates

- **AK Analyst Patterns**: Generated patterns for death rate analysis (2018-2021)
  - Pattern constraints
  - Feature relationships
  - County classifications

This notebook demonstrates the initial data processing phase of our analysis pipeline, focusing on preparing the data for subsequent clustering analysis.

#### Author : Drishti Singh


### Data Import and Setup
This section initializes the required libraries and sets up the Google Drive connection for data access.

In [1]:
import pandas as pd
import json
import math
import numpy as np
import csv

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Configuration Setup
Define source and destination paths for data processing. This ensures consistent file handling throughout the pipeline.

In [2]:
# Change file names here
source_folder = '/content/drive/MyDrive/AP Work/source/'
destination_folder = '/content/drive/MyDrive/AP Work/result-2/'

csv_file = source_folder + 'NORC_data_2017-2020.csv'
csv_file_pattern = source_folder +  'patterns_for_death-rate-2018-2021.csv'

### Feature Dictionary Creation
Create a mapping between features and their IDs. This step is crucial for maintaining consistent feature references throughout the analysis.

In [3]:
import os

destination_file = destination_folder + 'feature_dict.csv'

# Check if the directory exists, and create it if not
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Load the dataset
data = pd.read_csv(csv_file)

# Extract column names except for 'fips' (considering each column name as a feature)
feature_names = [col for col in data.columns if col != 'fips']

# Create a DataFrame with Feature_ID and Feature_Name
feature_dict = pd.DataFrame({
    'Feature_ID': [f'f{i+1}' for i in range(len(feature_names))],
    'Feature_Name': feature_names
})

feature_dict.to_csv(destination_file, index=False)

print(f"Feature dictionary saved to {destination_file}")

Feature dictionary saved to /content/drive/MyDrive/AP Work/result-2/feature_dict.csv


### Pattern Constraint Processing
Process the pattern constraints from the AK Analyst Platform output. This step maps patterns to their corresponding feature constraints.

In [4]:
import pandas as pd
import ast

# Load the feature dictionary
feature_dict_csv = destination_folder + 'feature_dict.csv'
feature_dict = pd.read_csv(feature_dict_csv)

dict_features = feature_dict.set_index('Feature_Name')['Feature_ID'].to_dict()

# Load the pattern data
patterns_df = pd.read_csv(csv_file_pattern)

# Initialize an empty list to store rows for the new CSV
output_rows = []

# Iterate over each row in the patterns DataFrame
for index, row in patterns_df.iterrows():

    # Parse the description column (which is a string representation of a dictionary)
    description_str = row['description']
    description_str = description_str.replace('inf', 'float("inf")').replace('-inf', 'float("-inf")')
    # print(description_str)
    description = eval(description_str)
    pattern_id = description['ID']

    # Get constraints dictionary
    constraints = description['constraints']

    # Iterate over each constraint (feature name and its bounds)
    for feature_name, bounds in constraints.items():
        # Map the feature name to its ID using dict_features
        if feature_name in dict_features:
            feature_id = dict_features[feature_name]

            # Extract upper bound (ub) and lower bound (lb)
            ub = bounds.get('ub', None)
            lb = bounds.get('lb', None)

            # Append a new row with Pattern_id, feature_id, ub, lb
            output_rows.append([pattern_id, feature_id, ub, lb])

# Create a DataFrame for output rows
output_df = pd.DataFrame(output_rows, columns=['Pattern_id', 'Feature_ID', 'UB', 'LB'])

# Save to CSV at the specified destination folder
destination_folder = '/content/drive/MyDrive/AP Work/result-2/'
output_df.to_csv(f'{destination_folder}/pattern_constraints.csv', index=False)

In [5]:
import pandas as pd
import numpy as np

source_folder = '/content/drive/MyDrive/AP Work/source/'
destination_folder = '/content/drive/MyDrive/AP Work/result-2/'

norc_data_path = source_folder + 'NORC_data_2017-2020.csv'
pattern_constraints_path = '/content/drive/MyDrive/AP Work/result-2/pattern_constraints.csv'
feature_dict_path = destination_folder + 'feature_dict.csv'

# Load CSV files
pattern_constraints = pd.read_csv(pattern_constraints_path)
norc_data = pd.read_csv(norc_data_path)
feature_dict = pd.read_csv(feature_dict_path)

# Create a dictionary to map Feature_ID to Feature_Name
feature_map = dict(zip(feature_dict['Feature_ID'], feature_dict['Feature_Name']))

# Initialize a list to store rows for the new CSV
output_rows = []

# Iterate over each pattern_id in pattern_constraints
for pattern_id in pattern_constraints['Pattern_id'].unique():
    # Get constraints for this pattern_id
    pattern_features = pattern_constraints[pattern_constraints['Pattern_id'] == pattern_id]

    # Filter counties in NORC data that satisfy all constraints for this pattern
    valid_counties = norc_data.copy()

    # Iterate over each feature in this pattern's constraints
    for _, row in pattern_features.iterrows():
        feature_name = feature_map[row['Feature_ID']]  # Map Feature_ID to Feature_Name
        lb, ub = row['LB'], row['UB']

        # Apply constraints to filter valid counties
        if pd.notna(lb):
            valid_counties = valid_counties[valid_counties[feature_name] >= lb]
        if pd.notna(ub):
            valid_counties = valid_counties[valid_counties[feature_name] <= ub]

    # Add valid counties with pattern_id to output_rows
    for _, county_row in valid_counties.iterrows():
        output_row = [pattern_id] + county_row.tolist()  # Add pattern_id followed by county data
        output_rows.append(output_row)

# Create a DataFrame fo   r the output rows
output_columns = ['Pattern_id'] + norc_data.columns.tolist()
output_df = pd.DataFrame(output_rows, columns=output_columns)

# Save the output DataFrame as a new CSV file
output_df.to_csv(f'{destination_folder}/norc_with_pattern.csv', index=False)

In [6]:
import pandas as pd

# Load the filtered_norc_with_pattern.csv file
filtered_norc_path = destination_folder + '/norc_with_pattern.csv'
filtered_norc = pd.read_csv(filtered_norc_path)

# Group by 'Pattern_id' and count the number of rows for each pattern_id
pattern_counts = filtered_norc['Pattern_id'].value_counts().reset_index()
pattern_counts.columns = ['Pattern_id', 'Row_Count']

# Calculate the total number of rows
total_rows = len(filtered_norc)

# Print the result
print(pattern_counts)
print(f"Total number of rows: {total_rows}")

    Pattern_id  Row_Count
0           62       2303
1           61       2275
2           60       2188
3           59       2137
4           58       2088
..         ...        ...
58           4         73
59           2         67
60           3         66
61           1         55
62           0         36

[63 rows x 2 columns]
Total number of rows: 33196


### County-Pattern Matching
Match counties to their corresponding patterns based on defined constraints. This creates the foundation for our clustering analysis.

In [7]:
import pandas as pd
import numpy as np

# File paths
source_folder = '/content/drive/MyDrive/AP Work/source/'
destination_folder = '/content/drive/MyDrive/AP Work/result-2/'
norc_data_path = source_folder + 'NORC_data_2017-2020.csv'
filtered_norc_path = destination_folder + 'norc_with_pattern.csv'

# Load CSV files
filtered_norc = pd.read_csv(filtered_norc_path)
norc_data = pd.read_csv(norc_data_path)

# Step 1: Handle 'Urbanicity' by converting 'Urban' to 1 and 'Rural' to 0
filtered_norc['Urbanicity'] = filtered_norc['Urbanicity'].map({'Urban': 1, 'Rural': 0})

# Step 2: Remove rows with missing values (NaN)
filtered_norc = filtered_norc.dropna()

# Step 3: Select numeric columns only for normalization (exclude 'Pattern_id' and 'fips')
numeric_columns = [col for col in filtered_norc.columns if pd.api.types.is_numeric_dtype(filtered_norc[col]) and col not in ['Pattern_id', 'fips']]

# Step 4: Min-Max normalization function
def min_max_normalize(df, columns):
    return df[columns].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Step 5: Normalize the selected numeric columns
filtered_norc[numeric_columns] = min_max_normalize(filtered_norc, numeric_columns)

# Step 6: Save the normalized DataFrame to a new CSV file
normalized_output_path = f'{destination_folder}/normalized_norc_with_pattern.csv'
filtered_norc.to_csv(normalized_output_path, index=False)

print(f"Normalized data saved to {normalized_output_path}")


<ipython-input-7-88da6bb12c98>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_norc[numeric_columns] = min_max_normalize(filtered_norc, numeric_columns)


Normalized data saved to /content/drive/MyDrive/AP Work/result-2//normalized_norc_with_pattern.csv


### Data Normalization
Normalize the dataset to ensure consistent feature scaling and handle categorical variables appropriately

In [8]:
import pandas as pd

def modify_data_based_on_pattern(filtered_norc_path, pattern_constraints_path, destination_folder, feature_dict_path):
    # Load the normalized filtered NORC data
    filtered_norc = pd.read_csv(filtered_norc_path)

    # Load pattern constraints that map Pattern_id to Feature_ID
    pattern_constraints = pd.read_csv(pattern_constraints_path)

    # Load the feature dictionary to map Feature_ID to Feature_Name
    feature_dict = pd.read_csv(feature_dict_path)
    feature_map = dict(zip(feature_dict['Feature_ID'], feature_dict['Feature_Name']))

    # Create a mapping of Pattern_id to the corresponding feature names
    pattern_feature_map = {}
    for _, row in pattern_constraints.iterrows():
        pattern_id = row['Pattern_id']
        feature_id = row['Feature_ID']
        feature_name = feature_map.get(feature_id, None)  # Get the feature name from Feature_ID
        if feature_name:  # Ensure that the feature name is valid
            if pattern_id not in pattern_feature_map:
                pattern_feature_map[pattern_id] = []
            pattern_feature_map[pattern_id].append(feature_name)

    # Step 1: Iterate over the rows of the filtered_norc dataset
    output_rows = []

    for _, row in filtered_norc.iterrows():
        pattern_id = row['Pattern_id']

        # Get the features associated with this pattern_id
        valid_features = pattern_feature_map.get(pattern_id, [])

        # Step 2: Create a new row where features not part of the valid features are set to -1
        new_row = row.copy()

        # For each column, set it to -1 if the feature isn't part of the current pattern's features
        for col in filtered_norc.columns:
            if col != 'Pattern_id' and col != 'fips' and col not in valid_features:
                new_row[col] = -1  # Set to -1 for features that are not part of the pattern

        output_rows.append(new_row)

    # Step 3: Create a new DataFrame from the modified rows
    modified_norc_df = pd.DataFrame(output_rows)

    # Step 4: Save the modified DataFrame to a new CSV file
    modified_norc_path = f'{destination_folder}/final.csv'
    modified_norc_df.to_csv(modified_norc_path, index=False)

    print(f"Modified data saved to {modified_norc_path}")


### Final Dataset Generation
Generate the final dataset by applying pattern-specific modifications and preparing the data for clustering analysis.

In [9]:
filtered_norc_path = '/content/drive/MyDrive/AP Work/result-2/normalized_norc_with_pattern.csv'
pattern_constraints_path = '/content/drive/MyDrive/AP Work/result-2/pattern_constraints.csv'
destination_folder = '/content/drive/MyDrive/AP Work/result-2/'
feature_dict_path = '/content/drive/MyDrive/AP Work/result-2/feature_dict.csv'

modify_data_based_on_pattern(filtered_norc_path, pattern_constraints_path, destination_folder, feature_dict_path)


Modified data saved to /content/drive/MyDrive/AP Work/result-2//final.csv


### Data Validation
Perform sanity checks on the processed data to ensure data quality and consistency.


In [10]:
import pandas as pd
import numpy as np

def sanity_check_clustering_data(normalized_norc_path, feature_dict_path):
    # Load the normalized filtered NORC data and feature dictionary
    normalized_norc = pd.read_csv(normalized_norc_path)
    feature_dict = pd.read_csv(feature_dict_path)

    # Get the list of feature names (excluding 'Pattern_id', 'fips', 'Urbanicity')
    feature_names = [col for col in normalized_norc.columns if col not in ['Pattern_id', 'fips', 'Urbanicity']]

    # Get a dictionary mapping feature names to their feature IDs
    feature_dict_map = dict(zip(feature_dict['Feature_Name'], feature_dict['Feature_ID']))

    # Initialize a list to store errors
    errors = []

    # Check for missing values
    missing_values = normalized_norc[normalized_norc.isnull().any(axis=1)]
    if not missing_values.empty:
        for _, row in missing_values.iterrows():
            errors.append((row['Pattern_id'], row['fips'], 'Missing value in row'))

    # Iterate through each row to check for issues
    for index, row in normalized_norc.iterrows():
        pattern_id = row['Pattern_id']
        fips = row['fips']

        # Check each feature (column) to see if it is normalized correctly (i.e., between 0 and 1)
        for feature_name in feature_names:
            # If the feature does not belong to the current pattern, check if its value is -1
            if pd.isna(row[feature_name]) or row[feature_name] < 0 or row[feature_name] > 1:
                if row[feature_name] != -1:
                    errors.append((pattern_id, fips, feature_name, row[feature_name]))

        # Check for duplicate columns in the dataset
        duplicate_columns = normalized_norc.columns[normalized_norc.columns.duplicated()]
        if not duplicate_columns.empty:
            errors.append((pattern_id, fips, 'Duplicate column', list(duplicate_columns)))

    # Print the errors if found
    if errors:
        print("Issues found in the following rows:")
        for error in errors:
            print(f"Pattern ID: {error[0]}, County ID (fips): {error[1]}, Feature: {error[2]}, Value: {error[3]}")
    else:
        print("Sanity check passed. No issues found.")

# Path to the normalized filtered NORC file and feature dictionary
normalized_norc_path = '/content/drive/MyDrive/AP Work/result-2/final.csv'
feature_dict_path = '/content/drive/MyDrive/AP Work/result-2/feature_dict.csv'

# Perform the sanity check for clustering
sanity_check_clustering_data(normalized_norc_path, feature_dict_path)


Sanity check passed. No issues found.
